In [1]:
import pandas as pd
import dateutil
import numpy as np
from era5_down import era5_down
import glob
import xarray as xr
import os
import re
import joblib
import geopandas as gpd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Improved function to sum dataframe columns which contain nan's
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

In [4]:
def xgb_predictions(X):
    predictions = {}
    for pkl_file in models:
        file_num = int(re.search(r'\d+', pkl_file).group())
        str_model = os.path.join(dir_ann, pkl_file)
        xgb = joblib.load(str_model)
        y_pred = xgb.predict(X)
        predictions[file_num] = y_pred

    new_df = pd.DataFrame(predictions)
    new_df = new_df[sorted(new_df.columns)]

    return new_df

In [5]:
def add_variable_along_timelatlon(ds, var, name, units, long_name):
    ds[name] = (('time','lat','lon'), var)
    ds[name].attrs['units'] = units
    ds[name].attrs['long_name'] = long_name
    return ds


In [6]:
t2m = 't2m'
rh2 = 'rh'
ff  = 'u2'
Prec = 'tp'
Snowfall = 'sf'
SWD = 'SWin'
LWD = 'LWin'
Pres = 'press'
fcc  = 'tcc'
msl  = 'msl'

In [7]:
files = sorted(glob.glob('../PDD_model_chris/data/input/ERA5/ERA_59_20_day/*.nc'))
len(files)

62

In [8]:
df_g = gpd.read_file('../PDD_model_chris/data/static/Shapefiles/SSI.shp')
ds   = xr.open_dataset('../PDD_model_chris/data/static/SSI_static_200.nc')

ERROR 1: PROJ: proj_create_from_database: Open of /home/christian/miniconda3/envs/DL_SMB/share/proj failed


In [9]:
Hurd = df_g[df_g.Name == 'Hurd Glacier']
Johnsons = df_g[df_g.Name == 'Johnsons Glacier']

In [10]:
bound = Hurd.total_bounds
min_lon = bound[0]
min_lat = bound[1]
max_lon = bound[2]
max_lat = bound[3]
mask_lon = (ds.lon >= min_lon) & (ds.lon <= max_lon)
mask_lat = (ds.lat >= min_lat) & (ds.lat <= max_lat)
cropped_hurd = ds.where(mask_lon & mask_lat, drop=True)
cropped_hurd

bound = Johnsons.total_bounds
min_lon = bound[0]
min_lat = bound[1]
max_lon = bound[2]
max_lat = bound[3]
mask_lon = (ds.lon >= min_lon) & (ds.lon <= max_lon)
mask_lat = (ds.lat >= min_lat) & (ds.lat <= max_lat)
cropped_Johnsons = ds.where(mask_lon & mask_lat, drop=True)
cropped_Johnsons

<xarray.Dataset>
Dimensions:  (lat: 16, lon: 10)
Coordinates:
  * lon      (lon) float64 -60.38 -60.37 -60.37 -60.36 ... -60.34 -60.33 -60.33
  * lat      (lat) float64 -62.69 -62.68 -62.68 -62.68 ... -62.66 -62.66 -62.66
Data variables:
    HGT      (lat, lon) float32 327.0 332.0 339.0 344.0 ... 152.0 167.0 203.0
    ASPECT   (lat, lon) float32 250.7 244.2 262.6 268.4 ... 279.1 269.8 264.9
    SLOPE    (lat, lon) float32 1.31 1.161 0.548 0.3874 ... 2.848 2.611 3.213
    MASK     (lat, lon) float32 1.0 1.0 1.0 nan nan nan ... 1.0 1.0 1.0 1.0 1.0

In [11]:
dso = cropped_Johnsons.copy()
dso.coords['time'] = pd.date_range('1959-01-01', '2020-12-31', freq='1M')

In [12]:
smb_dis     = np.full([len(dso.time), len(dso.lat), len(dso.lon)], np.nan)
accu_dis    = np.full([len(dso.time), len(dso.lat), len(dso.lon)], np.nan)
t2m_dis     = np.full([len(dso.time), len(dso.lat), len(dso.lon)], np.nan)
melt_dis    = np.full([len(dso.time), len(dso.lat), len(dso.lon)], np.nan)
runoff_dis  = np.full([len(dso.time), len(dso.lat), len(dso.lon)], np.nan)
MO_dis      = np.full([len(dso.time), len(dso.lat), len(dso.lon)], np.nan)

In [13]:
dir_ann = 'DL_SSI_glaciers/RF/LOYSO_ERA5/CV/'
models = sorted(os.listdir(dir_ann))

In [14]:
ii = 0
for i in range(len(dso.lat)):
    for j in range(len(dso.lon)):
        if (dso['MASK'][i, j].values == 1):
            print(ii)
            ii = ii + 1
            df_day = era5_down(files, 
                               dso['lon'][j].values, 
                               dso['lat'][i].values,
                               dso['HGT'][i, j].values)
            df_day['t2m_an'] = (df_day[t2m] - df_day[t2m].mean())/df_day[t2m].std()
            df_day['PDD']    = df_day[t2m].where(df_day[t2m] > 0, 0).where(df_day[t2m] <= 0, 1)
            df_month = df_day.resample('1M').agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                      SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                      Snowfall:nansumwrapper, msl:np.mean,
                                      Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                      'PDD':nansumwrapper})
            df_month['lon']  = dso['lon'][j].values
            df_month['lat']  = dso['lat'][i].values
            df_month['elev'] = dso['HGT'][i, j].values
            df_month_or = df_month[['lon', 'lat', 'elev', t2m, rh2, ff, SWD, LWD, 
                                    Prec, Snowfall, msl, Pres, fcc, 't2m_an', 'PDD']]
            X = df_month_or.to_numpy()
            df_smb = xgb_predictions(X)
            df_smb = df_smb.mean(axis=1).to_frame()
            df_month_or['smb']      = df_smb.values
            df_month_or['melt']     = df_month_or['smb'] - df_month_or[Snowfall]
            df_month_or['melt']     = np.abs(df_month_or['melt'].where(df_month_or['melt'] < 0, 0))
            MO = ((0.003 * df_month_or[t2m].values + 0.52)* df_month_or[Snowfall].values)
            df_month_or['MO']       = MO
            df_month_or['runoff']   = df_month_or['melt'] - df_month_or['MO']
            df_month_or['runoff']   = df_month_or['runoff'].where(df_month_or['runoff'] > 0, 0)
            
            t2m_dis[:, i, j]     = df_month_or[t2m].values
            accu_dis[:, i, j]    = df_month_or[Snowfall].values
            smb_dis[:, i, j]     = df_month_or['smb'].values
            melt_dis[:, i, j]    = df_month_or['melt'].values
            runoff_dis[:, i, j]  = df_month_or['runoff'].values
            MO_dis[:, i, j]      = df_month_or['MO'].values


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


In [15]:
add_variable_along_timelatlon(dso, t2m_dis, 'T2', '°C', 'Temperature at 2 m')
add_variable_along_timelatlon(dso, accu_dis, 'Accu', 'm w.e.', 'Accumulation')
add_variable_along_timelatlon(dso, smb_dis, 'smb', 'm w.e.', 'Surface Mass Balance')
add_variable_along_timelatlon(dso, melt_dis, 'Melt', 'm w.e.', 'Melt')
add_variable_along_timelatlon(dso, runoff_dis, 'Q', 'm w.e.', 'Runoff')
add_variable_along_timelatlon(dso, MO_dis, 'MO', 'm w.e.', 'Refreezing')

<xarray.Dataset>
Dimensions:  (lat: 16, lon: 10, time: 744)
Coordinates:
  * lon      (lon) float64 -60.38 -60.37 -60.37 -60.36 ... -60.34 -60.33 -60.33
  * lat      (lat) float64 -62.69 -62.68 -62.68 -62.68 ... -62.66 -62.66 -62.66
  * time     (time) datetime64[ns] 1959-01-31 1959-02-28 ... 2020-12-31
Data variables:
    HGT      (lat, lon) float32 327.0 332.0 339.0 344.0 ... 152.0 167.0 203.0
    ASPECT   (lat, lon) float32 250.7 244.2 262.6 268.4 ... 279.1 269.8 264.9
    SLOPE    (lat, lon) float32 1.31 1.161 0.548 0.3874 ... 2.848 2.611 3.213
    MASK     (lat, lon) float32 1.0 1.0 1.0 nan nan nan ... 1.0 1.0 1.0 1.0 1.0
    T2       (time, lat, lon) float64 -2.112 -2.18 -2.243 ... -0.7099 -1.034
    Accu     (time, lat, lon) float64 0.08073 0.08137 ... 0.04371 0.04929
    smb      (time, lat, lon) float64 0.06721 0.06858 ... -0.2227 -0.05971
    Melt     (time, lat, lon) float64 0.01352 0.01279 0.01462 ... 0.2664 0.109
    Q        (time, lat, lon) float64 0.0 0.0 0.0 nan ... 0.2483 0.2437 0.08352
    MO       (time, lat, lon) float64 0.04147 0.04178 ... 0.02263 0.02548

In [16]:
dso.to_netcdf('Johnsons_glacier.nc')